In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.SN_Curve import SN_Curve_fatpack, SN_Curve_qats
from utils.extract_and_preprocess_data import extract_and_preprocess_data
from utils.setup_custom_logger import setup_custom_logger
from utils.read_simulation_file import read_bladed_file
from utils.calculate_damage import find_stress_range_and_count_with_rainflow
from utils.save_damage_table import save_damage_table
from utils.create_geo_matrix import create_geo_matrix
import numpy as np
import sys
import pandas as pd
import fatpack 
from qats.fatigue.rainflow import count_cycles
from qats.fatigue.sn import SNCurve
from qats.fatigue.sn import minersum
import os 
import rainflow

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [30 / 2.54, 20 / 2.54]


In [3]:
# Get relevant data_paths for DLC and simulation result files
DLC_IDs = ['DLC12'] #, 'DLC64a', 'DLC64b', 'DLC24a', 'DLC31', 'DLC41a', 'DLC41b']
data_path              = fr'{os.getcwd()}\data'
DLC_file               = data_path +  r'\Doc-0081164-HAL-X-13MW-DGB-A-OWF-Detailed DLC List-Fatigue Support Structure Load Assessment_Rev7.0.xlsx'
sim_res_cluster_folder = data_path +  r'\Doc-0089427-HAL-X-13MW DB-A OWF-ILA3_JLO-model_fatigue_timeseries_all_elevations'
geometry_file          = data_path +  r'\DA_P53_CD.xlsx'

geometry      = pd.read_excel(geometry_file)# .drop([1,2]) # TODO drop can be used for testing only a set of geometries
n_geometries  = geometry.shape[0]
point_angles  = [float(i) for i in range(0,359,15)]
cluster_ID    = 'JLO'
curve         = SN_Curve_fatpack('D') # DNV SN curve in air, D
# curve         = SN_Curve_qats('D') #
out_file_type = 'mat' # or npy 
geo_matrix    = create_geo_matrix(geometry, point_angles, curve) # better matrix to pass to the main function
logger        = setup_custom_logger('Main') # logger.info('Message') and logger.error('Message')


In [4]:
DLC_ID = DLC_IDs[0]

In [5]:
def get_stress_range_and_count_rainflow_pckg(stress_timeseries, k = 128):
    '''
    Uses rainflow counting for finding stress ranges and their respective counts 
    '''
    ''' Returns a (N_ranges, 2) matrix with col0 = stress range and col1 = counts of given ranges.
    '''
    # Example of manual inspection
    # cycles = [(rng, mean, count, ix_range_start, ix_range_end) for rng, mean, count, ix_range_start, ix_range_end in rainflow.extract_cycles(stress_timeseries) ] 
    # ranges = np.array( [c_tup[0] for c_tup in cycles ] )
    # counts = np.array( [c_tup[2] for c_tup in cycles ] )
    
    # Returns a sorted list of (ranges, counts), where counts = 0.5 for half cycles. This can be used for scaling properly in the damage estimation. 
    return np.array(rainflow.count_cycles(stress_timeseries)) #, nbins = k))

In [6]:
def get_stress_range_and_count_qats_pckg(stress_timeseries, k = 128):
    '''
    Uses rainflow counting for finding stress ranges and their respective counts 
    '''
    # k is just a dummy to be compatible with the rainflow package's func
    r, m, c = count_cycles(stress_timeseries).T
    return np.hstack((r.reshape(-1,1),c.reshape(-1,1))) 

In [7]:
stress_range_finder_func_dict = {'rf': get_stress_range_and_count_rainflow_pckg, 'qats':get_stress_range_and_count_qats_pckg}

In [8]:
stress_range_finder_func = stress_range_finder_func_dict['rf']

In [9]:
df, probs, n_cases, n_timesteps = extract_and_preprocess_data(DLC_file, DLC_ID, cluster_ID, sim_res_cluster_folder)

In [10]:
def calculate_damage_case_i(binary_file_i, description_file_i, point_angles, geo_matrix, curve, stress_range_finder_func):
    
    content_reshaped = read_bladed_file(binary_file_i, description_file_i) # (n,m) numpy array with n = no. of quantities of data,  m = no. of timesteps for each data quantity 
    damage = np.zeros( (len(geo_matrix), len(point_angles)) )
    
    for geo_idx, geo_dict in geo_matrix.items():
        pos = (geo_dict['mx_col'] - 1, geo_dict['my_col'] - 1, geo_dict['fz_col'] - 1) # columns in DLC file to collect moments and forces, using -1 to fit Python indexing
        
        moments_x_timeseries = content_reshaped[[pos[0]], :] # Moments as (1, timesteps) array - hence the [[],:] type slice
        moments_y_timeseries = content_reshaped[[pos[1]], :] # Moments as (1, timesteps) array
        forces_z_case_i      = content_reshaped[[pos[2]], :] # Axial F as (1, timesteps) array 

        # Resultant moment for the current case -> (theta, timestep)-shaped array    
        actual_angles_rad = np.deg2rad(geo_dict['actual_angles'])
        res_moments_timeseries_case_i = np.sin([actual_angles_rad]).T.dot(moments_x_timeseries) + np.cos([actual_angles_rad]).T.dot(moments_y_timeseries)
        res_force_timeseries_case_i = np.repeat(forces_z_case_i, len(actual_angles_rad), axis=0)
                
        # size (theta, timestep)
        stress_timeseries_case_i = res_force_timeseries_case_i / geo_dict['A'] + res_moments_timeseries_case_i / geo_dict['Z'] # [Pa] Quick check: N / m**2 + Nm / m**3 = N / m**2 = Pa
        
        # Adjust the stress according the stress concentration factors for certain angles
        stress_timeseries_case_i *= np.array(geo_dict['scf_per_point'])[:, None] # Elementwise multiplication row wise 
        stress_timeseries_case_i *= geo_dict['alpha'] * 1e-6 # [MPa]
        
        # def calculate_damage_case_i(stress_timeseries_case_i, curve, stress_range_finder_func):
        for ang_idx, stress_timeseries_case_i_ang_j in enumerate(stress_timeseries_case_i):
            # stress_ranges comes as (N_stress_ranges, n_counts) sized matrix from rainflowpackage and qats
            
            stress_ranges = stress_range_finder_func(stress_timeseries_case_i_ang_j, k = 128)
            dmg = curve.miner_sum(stress_ranges)
            
            # print(stress_ranges)
            print(f'Found {stress_ranges.shape[0]} stress ranges with sum {stress_ranges.sum()}')
            print(f'First ranges: {stress_ranges[:5,0]}')
            print(f'Damage: {dmg}')
            return
            
            damage[geo_idx, ang_idx] = dmg 
    return damage # (n_geo, n_angles) shaped array

In [11]:
damage_table_DLC_i = np.zeros((n_cases, n_geometries, len(point_angles))) # pre-allocate output matrix of the current DLC

In [12]:
case_i_test = 0
curves = [SN_Curve_fatpack('D'), SN_Curve_qats('D')]
range_finders = ['rf', 'qats']
stress_range_finder_func_list = [get_stress_range_and_count_rainflow_pckg, get_stress_range_and_count_qats_pckg]


In [13]:
for c in curves:
    for iterator, func in enumerate(stress_range_finder_func_list):
        print(f'Rainflow method {range_finders[iterator]} with {c.SN}')
        
        arguments = [(df.results_files[i], 
                        df.descr_files[i], 
                        point_angles, 
                        geo_matrix, 
                        c,
                        func,
                        ) for i in range(n_cases)]
    
        calculate_damage_case_i(*arguments[case_i_test])

Rainflow method rf with <fatpack.endurance.BiLinearEnduranceCurve object at 0x000001E2E46957F0>
Found 135 stress ranges with sum 671.7668475134153
First ranges: [0.00555728 0.01632959 0.02042103 0.02722289 0.03667329]
Damage: 2.7719836298946203e-10
Rainflow method qats with <fatpack.endurance.BiLinearEnduranceCurve object at 0x000001E2E46957F0>
Found 133 stress ranges with sum 669.313970690127
First ranges: [0.00555728 0.01632959 0.02042103 0.02722289 0.03667329]
Damage: 2.7719812948793826e-10
Rainflow method rf with <SNCurve "qats SN curve, "D in air"" (bi-linear)>
Found 135 stress ranges with sum 671.7668475134153
First ranges: [0.00555728 0.01632959 0.02042103 0.02722289 0.03667329]
Damage: 2.7719836298946193e-10
Rainflow method qats with <SNCurve "qats SN curve, "D in air"" (bi-linear)>
Found 133 stress ranges with sum 669.313970690127
First ranges: [0.00555728 0.01632959 0.02042103 0.02722289 0.03667329]
Damage: 2.771981294879381e-10


In [14]:
from utils.SN_Curve import SN_Curve_fatpack, SN_Curve_qats
#curves[0].plot_characteristics()
#curves[1].plot_characteristics()
# print(curves[1].SN.print_parameters())

In [15]:
from calculate_lifetime_from_damage import create_overall_lookuptable

create_overall_lookuptable()

[[228.16152676 226.98603546 235.16161891 252.31363259 272.27436711
  284.10989571 282.48136958 272.47146289 260.50298698  42.26436317
  242.16404793  38.24347579 228.25696708 226.9436087  235.02462479
  252.10140817 272.00171456 283.77619896 282.05080158 271.98492826
  259.99797429 250.03282882 241.78764723 234.48238658]
 [266.74465405 265.33174169 274.85821807 295.07636996 318.30196025
  332.00188919 329.76510034 318.02880064 304.11487889 292.39537386
  282.86949047 274.31940921 266.86156514 265.27616455 274.68948611
  294.81901159 317.97196554 331.59813512 329.245949   317.44420326
  303.51078311 291.85663675 282.41423762 273.99820761]
 [426.78090808 424.72403734 440.25093605 472.94727641 509.35180732
  529.72791279 525.60646974 506.09596527 484.49478679 466.44643828
  451.75602098 438.54259717 427.0498214  424.62588463 439.92107699
  472.42169715 508.65684819 528.89401707 524.53205249 504.89487244
  483.26167627 465.28128432 450.80657077 437.86100993]]
38.243475791142046
       Nace

In [ ]:
arguments = [(df.results_files[i], 
              df.descr_files[i], 
              point_angles, 
              geo_matrix, 
              curve,
              stress_range_finder_func
              ) for i in range(n_cases)]
for case_i in range(n_cases):
    damage_table_DLC_i[[case_i], :, :] = calculate_damage_case_i(*arguments[case_i])

135
2.7719836298946203e-10


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [ ]:
# Transform the damage table into a combined_damage matrix of size (n_geometries, n_angles), weighting each of the cases by their probabilities and converting to hour-based damage    
combined_damage_DLC_i = np.zeros((n_geometries, len(point_angles)))
weights = np.array([probs])
for ang_idx in range(len(point_angles)):
    combined_damage_DLC_i[:, [ang_idx]] = np.dot( weights, damage_table_DLC_i[:,:, ang_idx]).T * 6.0 # (n_geometries, 1) -> multiplication of weights by dot product
     

In [ ]:

print(damage_table_DLC_i.shape)
print(damage_table_DLC_i)    